In [2]:
import numpy as np 
# READING AND PREPROCESISNGT THE TEXT FILE
with open('1268-0.txt', 'r',encoding="utf8") as f:
    text = f.read()


start_indx=text.find("THE MYSTERIOUS ISLAND")
end_indx=text.find("End of the Project Gutenberg EBook of The Mysterious Island, by Jules Verne")

text=text[start_indx:end_indx]
char_set=set(text)

print("Total number of characters in the text: ",len(text))
print("Total number of unique characters in the text: ",len(char_set))

Total number of characters in the text:  1130711
Total number of unique characters in the text:  85


In [3]:
chars_sorted =sorted(char_set)
char2int = {ch:i for i,ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)
text_encoded = np.array([char2int[ch] for ch in text],dtype=np.int32)
print ("The encoded shape : ",text_encoded.shape)
print("The encoded text is : ",text_encoded[:100])   
print("The decoded text is : ","".join(char_array[text_encoded[:100]]))


The encoded shape :  (1130711,)
The encoded text is :  [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1 37 47 40 29 42 32  1 10 10
 10  0  0  0  0  0 48 36 33  1 41 53 47 48 33 46 37 43 49 47  1 37 47 40
 29 42 32  0  0 56 79  1 38 75 66 59 73  1 50 59 72 68 59  0  0 16 23 22
 19  0  0  0  0  0 44 29 46 48  1 16 12 12 32 46 43 44 44 33 32  1 34 46
 43 41  1 48]
The decoded text is :  THE MYSTERIOUS ISLAND ***




THE MYSTERIOUS ISLAND

by Jules Verne

1874




PART 1--DROPPED FROM T


In [4]:
import torch 
from torch.utils.data import Dataset
seq_length = 40
chunk_size = seq_length + 1

text_chunks = [
    text_encoded[i:i+chunk_size] for i in range(len(text_encoded)-chunk_size+1)
]

from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self,text_chunks):
        self.text_chunks = text_chunks

    def __len__(self):
        return len(self.text_chunks)
    
    def __getitem__(self,idx):
        text_chunk = self.text_chunks[idx]
        return text_chunk[:-1].long(),text_chunk[1:].long()


seq_dataset = TextDataset(torch.tensor(text_chunks))

for i, (seq, target) in enumerate(seq_dataset):
    print("Sequence: ", i)
    print("Input: ", seq)
    print("Target: ", target)
    print("Input decoded: ", "".join(char_array[seq]))
    print("Target decoded: ", "".join(char_array[target]))
    if i == 5:
        break






Sequence:  0
Input:  tensor([48, 36, 33,  1, 41, 53, 47, 48, 33, 46, 37, 43, 49, 47,  1, 37, 47, 40,
        29, 42, 32,  1, 10, 10, 10,  0,  0,  0,  0,  0, 48, 36, 33,  1, 41, 53,
        47, 48, 33, 46])
Target:  tensor([36, 33,  1, 41, 53, 47, 48, 33, 46, 37, 43, 49, 47,  1, 37, 47, 40, 29,
        42, 32,  1, 10, 10, 10,  0,  0,  0,  0,  0, 48, 36, 33,  1, 41, 53, 47,
        48, 33, 46, 37])
Input decoded:  THE MYSTERIOUS ISLAND ***




THE MYSTER
Target decoded:  HE MYSTERIOUS ISLAND ***




THE MYSTERI
Sequence:  1
Input:  tensor([36, 33,  1, 41, 53, 47, 48, 33, 46, 37, 43, 49, 47,  1, 37, 47, 40, 29,
        42, 32,  1, 10, 10, 10,  0,  0,  0,  0,  0, 48, 36, 33,  1, 41, 53, 47,
        48, 33, 46, 37])
Target:  tensor([33,  1, 41, 53, 47, 48, 33, 46, 37, 43, 49, 47,  1, 37, 47, 40, 29, 42,
        32,  1, 10, 10, 10,  0,  0,  0,  0,  0, 48, 36, 33,  1, 41, 53, 47, 48,
        33, 46, 37, 43])
Input decoded:  HE MYSTERIOUS ISLAND ***




THE MYSTERI
Target decoded:  E MYSTERIOU

/var/folders/x7/f3fqnbfs4vg005y3fj658pd40000gn/T/ipykernel_7075/4240554814.py:24: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  seq_dataset = TextDataset(torch.tensor(text_chunks))


In [5]:
from torch.utils.data import DataLoader
batch_size = 64
torch.manual_seed(1)
seq_dl = DataLoader(seq_dataset,batch_size=batch_size,shuffle=True, drop_last=True)


In [6]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embed_dim)
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.LSTM(embed_dim,rnn_hidden_size,batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size,vocab_size)


    def forward(self,x,hidden, cell):
        out =  self.embedding(x).unsqueeze(1)
        out, (hidden, cell) = self.rnn(out,(hidden,cell))
        out = self.fc(out).reshape(out.size(0),-1)
        return out, hidden,cell
    def init_hidden(self,batch_size):
        hidden =torch.zeros(1,batch_size,self.rnn_hidden_size)
        cell = torch.zeros(1,batch_size,self.rnn_hidden_size)
        return  hidden, cell
    

vocab_size = len(char_array)
embed_dim = 256
rnn_hidden_size = 512
torch.manual_seed(1)
model = RNN(vocab_size,embed_dim,rnn_hidden_size)
model

RNN(
  (embedding): Embedding(85, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=85, bias=True)
)

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.005)

In [8]:
num_epochs = 10000
torch.manual_seed(1)
for epoch in range(num_epochs):
    hidden, cell = model.init_hidden(batch_size)
    seq_batch, target_batch = next(iter(seq_dl))
    optimizer.zero_grad()
    loss = 0.0
    for c in range(seq_length):
        output, hidden, cell = model(seq_batch[:,c],hidden,cell)
        loss += loss_fn(output,target_batch[:,c])
    loss.backward()
    optimizer.step()
    loss = loss.item()/seq_length
    if epoch % 500 == 0:
        print(f"Epoch {epoch} Loss {loss}")

Epoch 0 Loss 4.4364269256591795
Epoch 500 Loss 1.3617974281311036
Epoch 1000 Loss 1.2456331253051758
Epoch 1500 Loss 1.233212947845459
Epoch 2000 Loss 1.2209564208984376
Epoch 2500 Loss 1.1708856582641602
Epoch 3000 Loss 1.2241089820861817
Epoch 3500 Loss 1.1715151786804199
Epoch 4000 Loss 1.1603304862976074
Epoch 4500 Loss 1.1339111328125
Epoch 5000 Loss 1.131721591949463
Epoch 5500 Loss 1.152293872833252
Epoch 6000 Loss 1.1495393753051757
Epoch 6500 Loss 1.104940700531006
Epoch 7000 Loss 1.161044216156006
Epoch 7500 Loss 1.187358570098877
Epoch 8000 Loss 1.1354199409484864
Epoch 8500 Loss 1.1028923034667968
Epoch 9000 Loss 1.1335047721862792
Epoch 9500 Loss 1.1468483924865722


In [9]:
from torch.distributions.categorical import Categorical
def sample(model, starting_str, len_generated_text=500, scale_factor=1.0):
    encoded_input = torch.tensor([[char2int[ch] for ch in starting_str]])
    encoded_input = torch.reshape(encoded_input, (1, -1))
    generated_str = starting_str
    model.eval()
    hidden, cell = model.init_hidden(1)
    hidden, cell = model.init_hidden(1)
    for c in range(len(starting_str)-1):
        _, hidden, cell = model(
        encoded_input[:, c].view(1), hidden, cell
        )

    last_char = encoded_input[:, -1]
    for i in range(len_generated_text):
        logits, hidden, cell = model(
        last_char.view(1), hidden, cell
        )
        logits = torch.squeeze(logits, 0)
        scaled_logits = logits * scale_factor
        m = Categorical(logits=scaled_logits)
        last_char = m.sample()
        generated_str += str(char_array[last_char])

    return generated_str


In [10]:
torch.manual_seed(1)
print(sample(model, "The island", 500, scale_factor=1.0))

The island is singular enough a difficulty.
Cyrus Harding, as may derivain, and,
the light dured a molluscer from theim, disputing out obtain,
and the lad, viecial times of December,
sweal of vessity, the two hung
on them.”

“Ah it even eight o’clock,” replied the reporter, whose scriences were to be doubled along the wood. Two her titude one of the sea as we to near him, as he
would see, which appeared under which their sticks, and it was in wild exceed firm, and you tell now?”

“And later, that everythi


In [11]:
torch.save(model.state_dict(), "rnn_model.pth")